In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt

# Cost Function

In [4]:
def class_cost(c=1, delta=0.5, N=100):
    return c*((1-delta)*(1/(N+1)) + delta)

lims = {
    'delta1': [0, 1],
    'delta2': [0, 1],
    'c1': [0, 10],
    'c2': [0, 10],
    'N1': [1, 100],
    'N2': [1, 100],
}

@ipywidgets.interact_manual(
          plot_against=list(lims.keys()),
          delta1=(0., 1.),
          delta2=(0., 1.),
          c1=(0, 10),
          c2=(0, 10),
          N1=(1, 300),
          N2=(1, 300),
)
def plot(delta1=0.5, 
         delta2=0.5, 
         c1=1., 
         c2=1., 
         N1=100, 
         N2=100,
         plot_against='delta2'):
    vars = {
        'delta1': delta1,
        'delta2': delta2,
        'c1': c1,
        'c2': c2,
        'N1': N1,
        'N2': N2,
    }
    vars[plot_against] = np.linspace(lims[plot_against][0], lims[plot_against][1], 100)
    _, ax = plt.subplots(1, 1, figsize=(8, 6))
    J = class_cost(vars['c1'], vars['delta1'], vars['N1']) + class_cost(vars['c2'], vars['delta2'], vars['N2'])
    ax.plot(vars[plot_against], J)
    ax.set_ylabel('Cost J')
    ax.set_xlabel(plot_against)

interactive(children=(FloatSlider(value=0.5, description='delta1', max=1.0), FloatSlider(value=0.5, descriptio…